In [29]:
from decimal import ROUND_HALF_UP, Decimal

import numpy as np
import pandas as pd

In [3]:
df_price = pd.read_csv('../input/train_files/stock_prices.csv')
df_financial = pd.read_csv('../input/train_files/financials.csv')
df_stocklist = pd.read_csv('../input/stock_list.csv')

C:\Users\ntj_0\AppData\Local\Temp\ipykernel_16888\3882798696.py:2: DtypeWarning: Columns (14,15,16,17,18,19,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df_financial = pd.read_csv('../input/train_files/financials.csv')


In [27]:
def adjust_price(price):
    """
    Args:
        price (pd.DataFrame)  : pd.DataFrame include stock_price
    Returns:
        price DataFrame (pd.DataFrame): stock_price with generated AdjustedClose
    """
    # transform Date column into datetime
    price.loc[: ,"Date"] = pd.to_datetime(price.loc[: ,"Date"], format="%Y-%m-%d")

    def generate_adjusted_close(df):
        
        # sort data to generate CumulativeAdjustmentFactor
        df = df.sort_values("Date", ascending=False)
        # generate CumulativeAdjustmentFactor
        df.loc[:, "CumulativeAdjustmentFactor"] = df["AdjustmentFactor"].cumprod()
        # generate AdjustedClose
        df.loc[:, "AdjustedClose"] = (
            df["CumulativeAdjustmentFactor"] * df["Close"]
        ).map(lambda x: float(
            Decimal(str(x)).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP)
        ))
        # reverse order
        df = df.sort_values("Date")
        # to fill AdjustedClose, replace 0 into np.nan
        df.loc[df["AdjustedClose"] == 0, "AdjustedClose"] = np.nan
        # forward fill AdjustedClose
        df.loc[:, "AdjustedClose"] = df.loc[:, "AdjustedClose"].ffill()
        return df

    # generate AdjustedClose
    price = price.sort_values(["SecuritiesCode", "Date"])
    price = price.groupby("SecuritiesCode").apply(generate_adjusted_close).reset_index(drop=True)

    # price.set_index("Date", inplace=True)
    return price

In [12]:
price_cols = ['Close']

df_price[price_cols] = df_price[price_cols].replace(0.0, np.nan)

# add high/low
df_price["diff_high_low"] = (
    np.log1p(df_price["High"]) / np.log1p(df_price["Low"])
    ).astype("float16")
    
log_price = df_price[price_cols].apply(np.log1p)
diff_log_price = log_price.diff()
diff_log_price['SecuritiesCode'] = df_price.SecuritiesCode

# add new features
fea_price = [df_price]

group_code = df_price.groupby("SecuritiesCode")
price_gb_code = df_price.groupby("SecuritiesCode")[price_cols]
diff_log_price_gb_code = diff_log_price.groupby("SecuritiesCode")[price_cols]

for span in [20, 40, 60, 120]:
    pct_change = (
        price_gb_code.pct_change(span)
            .add_prefix(f"feat_pct{span}_")
            .astype('float16')
    )
    volatility = (
        diff_log_price_gb_code.rolling(span, min_periods=1)
            .std()
            .reset_index("SecuritiesCode", drop=True)
    )
    volatility = (
        (log_price / volatility).add_prefix(f"feat_vl{span}_")
            .astype('float16')
    )
    ma = (
        price_gb_code.rolling(span, min_periods=1)
            .mean()
            .reset_index("SecuritiesCode", drop=True)
    )
    ma = (df_price[price_cols] / ma).add_prefix(f"feat_rmr{span}_").astype('float16')
    diff_roll = (
        group_code.diff_high_low
            .rolling(span, min_periods=1)
            .mean().
            rename(f"feat_wd{span}")
            .reset_index("SecuritiesCode", drop=True)
            .astype('float16')
    )
    fea_price += [pct_change, volatility, ma, diff_roll]

df_price_fe = pd.concat(fea_price, axis=1)
df_price_fe = adjust_price(df_price_fe) # add adjusted close value
